<a href="https://colab.research.google.com/github/yash-jain-1/BERT4Crypto/blob/main/Cryptocurrency_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Augmentation

## Importing dataset from Kaggle

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("omershafiq/bitcoin-network-transactional-metadata")

print("Path to dataset files:", path)

100%|██████████| 1.76G/1.76G [00:21<00:00, 89.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/omershafiq/bitcoin-network-transactional-metadata/versions/1


In [3]:
import pandas as pd

# File paths for input and output
input_file = path+"/DG_out.csv"  # Replace with the actual file path
output_file_1 = '/content/first_half_dataset.csv'
output_file_2 = '/content/second_half_dataset.csv'

# Initialize counters
row_count = 0
rows_in_first_half = 0

# Calculate the total number of rows without loading the entire file into memory
for chunk in pd.read_csv(input_file, chunksize=10**6):  # 1 million rows at a time (adjust if needed)
    row_count += len(chunk)

# Calculate the number of rows in the first half
rows_in_first_half = row_count // 2

# Now, write the first half and second half to separate files
first_half_written = False
row_counter = 0

# Read and process the file in chunks
for chunk in pd.read_csv(input_file, chunksize=10**6):
    if not first_half_written:
        # If the chunk exceeds the first half limit, split it
        if row_counter + len(chunk) > rows_in_first_half:
            # Split the chunk into two parts
            first_part = chunk.iloc[:rows_in_first_half - row_counter]
            second_part = chunk.iloc[rows_in_first_half - row_counter:]

            # Save the first half and append to the file
            first_part.to_csv(output_file_1, mode='a', index=False, header=(row_counter == 0))

            # Save the second part to the second file
            second_part.to_csv(output_file_2, mode='a', index=False, header=True)

            first_half_written = True
        else:
            # Keep writing to the first half
            chunk.to_csv(output_file_1, mode='a', index=False, header=(row_counter == 0))
    else:
        # Write remaining chunks to the second half
        chunk.to_csv(output_file_2, mode='a', index=False, header=(row_counter == rows_in_first_half))

    row_counter += len(chunk)

print("Dataset successfully split and saved!")


Dataset successfully split and saved!


In [11]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle

In [7]:
df_full = pd.read_csv("/content/first_half_dataset.csv")

In [8]:
df_full = df_full.drop(["tx_hash","in_malicious","out_malicious","is_malicious","all_malicious"],axis = 1)
df_full.head()

,indegree,outdegree,in_btc,out_btc,total_btc,mean_in_btc,mean_out_btc,out_and_tx_malicious
0,0,1,0.0,50.0,50.0,0.0,50.0,0
1,1,2,50.0,50.0,100.0,50.0,25.0,0
2,1,1,10.0,10.0,20.0,10.0,10.0,0
3,1,1,40.0,30.0,70.0,40.0,30.0,0
4,1,2,30.0,30.0,60.0,30.0,15.0,0


In [12]:
df_full = shuffle(df_full, random_state=42)
df_full

,indegree,outdegree,in_btc,out_btc,total_btc,mean_in_btc,mean_out_btc,out_and_tx_malicious
13186494,1,1,0.010000,0.009000,0.019000,0.010000,0.009000,0
2181963,1,2,13.000000,13.000000,26.000000,13.000000,6.500000,0
1120332,1,2,4.418648,4.418648,8.837295,4.418648,2.209324,0
5165043,1,2,0.029585,0.029085,0.058669,0.029585,0.014542,0
813515,1,1,102.035515,102.025515,204.061030,102.035515,102.025515,0
...,...,...,...,...,...,...,...,...
4304572,1,1,0.010000,0.009500,0.019500,0.010000,0.009500,0
10081351,1,2,0.010000,0.009000,0.019000,0.010000,0.004500,0
6550634,1,2,0.029915,0.029415,0.059330,0.029915,0.014708,0
6423388,3,2,0.035802,0.035302,0.071104,0.011934,0.017651,0


In [13]:
count = 0
for i in df_full["out_and_tx_malicious"]:
    if i == 1:
        count+=1
print(count)

87


# Model

In [14]:
!pip install scikeras

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense
#from keras.wrappers.scikit_learn import KerasClassifier # This import is no longer valid
from scikeras.wrappers import KerasClassifier # Import KerasClassifier from scikeras instead
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from tensorflow.keras.layers import Dense, Dropout

In [16]:
df = df_full

In [17]:
df

,indegree,outdegree,in_btc,out_btc,total_btc,mean_in_btc,mean_out_btc,out_and_tx_malicious
13186494,1,1,0.010000,0.009000,0.019000,0.010000,0.009000,0
2181963,1,2,13.000000,13.000000,26.000000,13.000000,6.500000,0
1120332,1,2,4.418648,4.418648,8.837295,4.418648,2.209324,0
5165043,1,2,0.029585,0.029085,0.058669,0.029585,0.014542,0
813515,1,1,102.035515,102.025515,204.061030,102.035515,102.025515,0
...,...,...,...,...,...,...,...,...
4304572,1,1,0.010000,0.009500,0.019500,0.010000,0.009500,0
10081351,1,2,0.010000,0.009000,0.019000,0.010000,0.004500,0
6550634,1,2,0.029915,0.029415,0.059330,0.029915,0.014708,0
6423388,3,2,0.035802,0.035302,0.071104,0.011934,0.017651,0


In [18]:
X = df.drop(["out_and_tx_malicious"], axis=1)
y = df["out_and_tx_malicious"]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
def create_feedforward_nn(input_dim):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

base_model1 = create_feedforward_nn(X_train.shape[1])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def create_mlp(input_dim):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

base_model2 = create_mlp(X_train.shape[1])

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def create_lstm(input_shape, sequence_length):
    model = Sequential()
    model.add(LSTM(64, input_shape=(X_train.shape[1],1), return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# You need to reshape your X_train and X_test to be sequences with the appropriate sequence_length.
sequence_length = 10  # Adjust as needed based on your data
base_model3 = create_lstm(input_shape=X_train.shape[1], sequence_length=sequence_length)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [23]:
base_model1 = KerasClassifier(build_fn=create_feedforward_nn, input_dim=X_train.shape[1], epochs=10, batch_size=32, verbose=0)


In [26]:
base_model2 = KerasClassifier(build_fn=create_mlp, input_dim=X_train.shape[1], epochs=10, batch_size=32, verbose=0)


In [27]:
base_model3 = KerasClassifier(build_fn=create_lstm, input_shape=(X_train.shape[1], 1), sequence_length=10, epochs=10, batch_size=32, verbose=0)


In [28]:
meta_model = LogisticRegression()

In [29]:
stacking_model = StackingClassifier(estimators=[
    ('model1', base_model1),
    ('model2', base_model2),
    ('model3', base_model3)
], final_estimator=LogisticRegression())

In [ ]:
stacking_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
y_pred = stacking_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the ensemble model: {accuracy:.10f}")

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
recall = recall_score(y_test,y_pred)
print("precision = ", recall)

In [ ]:
import matplotlib.pyplot as plt

# Define the models and their corresponding metrics
models = ['Random Forest', 'MLP', 'Att LSTM', 'Deep Ensemble']
accuracy = [61.00, 95.00, 97.00, 99.63]
precision = [84.63, 96.55, 98.26, 99.40]
recall = [23.08, 89.99, 98.24, 99.82]

# Create a bar graph
plt.figure(figsize=(10, 6))
bar_width = 0.2
index = range(len(models))

plt.bar(index, accuracy, bar_width, label='Accuracy', color='b', align='center')
plt.bar([i + bar_width for i in index], precision, bar_width, label='Precision', color='g', align='center')
plt.bar([i + 2*bar_width for i in index], recall, bar_width, label='Recall', color='r', align='center')

# Set the labels and title
plt.xlabel('Models')
plt.ylabel('Metrics (%)')
plt.title('Performance Metrics for Different Models')
plt.xticks([i + bar_width for i in index], models)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()